# Descrizione

Nell'esperimento è stato utilizzato il modello AlexNet per la classificazione binaria delle sottoclassi Target. Non viene usato alcun meccanismo per bloccare autonomamente il treaning del modello, eseguendo tutte e 50 le epoche. Il PreProcessing è invariato.

Librerie

In [1]:
import torch.nn as nn
import torch.optim as optim
import glob
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torch.cuda.amp import GradScaler, autocast
import os
import csv
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import torch
import pandas as pd
import matplotlib.pyplot as plt

Inizializzazione dei parametri

In [2]:
# Percorsi dei dataset di addestramento, validazione e test
train_dataset_path = '/Users/massimo/PycharmProjects/UnderwaterSoundsClassification/Classificazione_Multiclasse/Training_Balanced_Target'

val_dataset_path = '/Users/massimo/PycharmProjects/UnderwaterSoundsClassification/Classificazione_Multiclasse/Evaluation_Balanced_Target'

test_dataset_path = '/Users/massimo/PycharmProjects/UnderwaterSoundsClassification/Classificazione_Multiclasse/Testing_Balanced_Target'

# Percorso della directory per salvare i checkpoint
checkpoint_dir = '/Users/massimo/PycharmProjects/UnderwaterSoundsClassification/Esperimento 1 Multiclasse/AlexNet'

# Numero di epoche di addestramento
num_epochs = 50

# Dimensione del batch per l'addestramento e la validazione
batch_size = 128

# Tasso di apprendimento per l'ottimizzatore
learning_rate = 0.001

# Numero di passaggi per accumulare gradienti prima di aggiornare i pesi
gradient_accumulation_steps = 4

# Definizione della trasformazione per il preprocessing delle immagini
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Nome del file CSV in cui salvare le metriche di addestramento
metrics_path = os.path.join(checkpoint_dir, 'training_metrics.csv')

Definizione della Classe CustomDataset

In [3]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d))])
        self.data = []
        self.targets = []

        for idx, cls in enumerate(self.classes):
            class_dir = os.path.join(self.root_dir, cls)
            for file in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file)
                if os.path.isfile(file_path):
                    self.data.append(file_path)
                    self.targets.append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        target = self.targets[idx]

        if self.transform:
            image = self.transform(image)

        return image, target

Funzioni per il Caricamento e il Salvataggio dei Checkpoint

In [4]:
def load_checkpoint(checkpoint_path, model, optimizer):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    start_epoch += 1
    metrics = checkpoint['metrics']
    return model, optimizer, start_epoch, metrics

def save_checkpoint(checkpoint_state, checkpoint_path):
    torch.save(checkpoint_state, checkpoint_path)

Funzione per il Calcolo delle Metriche

In [5]:
def calculate_metrics(loader, model, device):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Calcolo metriche"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return accuracy, precision, recall, f1

Preparazione dei Dati

In [6]:
from Dataset import CustomDataset

train_dataset = CustomDataset(train_dataset_path, transform=transform)
val_dataset = CustomDataset(val_dataset_path, transform=transform)

print(f"Classi trovate nel dataset di addestramento: {train_dataset.classes}")
print(f"Classi trovate nel dataset di validazione: {val_dataset.classes}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

Inizializzazione del Modello

In [7]:
num_classes = len(train_dataset.classes)
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

for param in model.features.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.backends.cudnn.benchmark = True
scaler = GradScaler()


Caricamento del Checkpoint

In [8]:
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

latest_checkpoint = glob.glob(os.path.join(checkpoint_dir, 'checkpoint_epoch_*.pth'))
if latest_checkpoint:
    latest_checkpoint = max(latest_checkpoint, key=os.path.getctime)
    model, optimizer, start_epoch, metrics = load_checkpoint(latest_checkpoint, model, optimizer)
else:
    model, optimizer, start_epoch, metrics = model, optimizer, 0, []

Ciclo di Addestramento e Valutazione

In [9]:
header_written = False
if start_epoch >= num_epochs:
        print(f"Modello già addestrato fino all'epoca {num_epochs}.")
else:
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_train_loss = 0.0
        optimizer.zero_grad()
        pbar_train = tqdm(train_loader, desc=f"Epoca {epoch + 1}/{num_epochs} - Addestramento")
        for step, (inputs, labels) in enumerate(pbar_train):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with autocast():
                outputs = model(inputs)
                train_loss = criterion(outputs, labels)
            scaler.scale(train_loss).backward()

            # Accumula gradienti
            if (step + 1) % gradient_accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_train_loss += train_loss.item()
            pbar_train.set_postfix({'Loss': running_train_loss / (step + 1)})

        # Valutazione del modello ad ogni epoca
        model.eval()
        running_val_loss = 0.0
        pbar_val = tqdm(val_loader, desc=f"Epoca {epoch + 1}/{num_epochs} - Valutazione")
        for inputs, labels in pbar_val:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            with torch.no_grad():
                with autocast():
                    outputs = model(inputs)
                    val_loss = criterion(outputs, labels)
                running_val_loss += val_loss.item()
                pbar_val.set_postfix({'Val Loss': running_val_loss / len(val_loader)})

        # Calcola le metriche di addestramento e validazione
        train_accuracy, train_precision, train_recall, train_f1 = calculate_metrics(train_loader, model, device)
        val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_loader, model, device)
        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_val_loss = running_val_loss / len(val_loader)

        # Stampa le metriche
        print(f"Epoca {epoch + 1}/{num_epochs} - "
              f"Loss Addestramento: {epoch_train_loss:.4f} - Acc Addestramento: {train_accuracy:.4f} - "
              f"Precision Addestramento: {train_precision:.4f} - Recall Addestramento: {train_recall:.4f} - "
              f"F1 Addestramento: {train_f1:.4f} - "
              f"Loss Valutazione: {epoch_val_loss:.4f} - Acc Valutazione: {val_accuracy:.4f} - "
              f"Precision Valutazione: {val_precision:.4f} - Recall Valutazione: {val_recall:.4f} - F1 Valutazione: {val_f1:.4f}")

        # Aggiorna il file CSV con le metriche
        with open(metrics_path, 'a', newline='') as csvfile:
            fieldnames = ['epoch', 'train_loss', 'train_accuracy', 'train_precision', 'train_recall', 'train_f1',
                          'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Scrittura dell'header solo se non è stato già scritto
            if not header_written:
                writer.writeheader()
                header_written = True

            # Scrittura delle metriche per l'epoca corrente
            writer.writerow({
                "epoch": epoch + 1,
                "train_loss": epoch_train_loss,
                "train_accuracy": train_accuracy,
                "train_precision": train_precision,
                "train_recall": train_recall,
                "train_f1": train_f1,
                "val_loss": epoch_val_loss,
                "val_accuracy": val_accuracy,
                "val_precision": val_precision,
                "val_recall": val_recall,
                "val_f1": val_f1,
            })

        # Salva il checkpoint
        checkpoint_state = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'metrics': metrics
        }
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + 1}.pth')
        save_checkpoint(checkpoint_state, checkpoint_path)
        print(f"Checkpoint salvato in: {checkpoint_path}")

Salvataggio del Modello Finale

In [ ]:
model_path = '/Users/massimo/PycharmProjects/UnderwaterSoundsClassification/Esperimento 1 Multiclasse/AlexNet/alexnet_model_multiclass_classification.pth'
torch.save(model.state_dict(), model_path)

print(f"Modello finale salvato in: {model_path}")

Creazione dei grafici di Addestramento e Validazione

In [10]:
df = pd.read_csv(os.path.join(checkpoint_dir, 'training_metrics.csv'))

# Estrai le metriche di addestramento e validazione
train_metrics = ['train_loss', 'train_accuracy', 'train_precision', 'train_recall', 'train_f1']
val_metrics = ['val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']

# Crea un grafico separato per ogni coppia di metriche
for train_metric, val_metric in zip(train_metrics, val_metrics):
    plt.figure(figsize=(10, 6))
    plt.plot(df['epoch'], df[train_metric], label='Train ' + train_metric.split('_')[1].capitalize())
    plt.plot(df['epoch'], df[val_metric], label='Validation ' + val_metric.split('_')[1].capitalize())
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.title(train_metric.split('_')[1].capitalize() + ' Comparison')
    plt.legend()
    plt.grid(True)
    plt.show()

Testing

In [2]:
def load_model(model_path, num_classes, device):
    model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dataset = CustomDataset(test_dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(test_dataset.classes)
model = load_model(model_path, num_classes, device)
model.eval()

criterion = nn.CrossEntropyLoss()
running_test_loss = 0.0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(inputs)
        test_loss = criterion(outputs, labels)
        running_test_loss += test_loss.item()

test_loss = running_test_loss / len(test_loader)
test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(test_loader, model, device)

# Creazione del file CSV nella directory dei checkpoint
csv_output_path = os.path.join(checkpoint_dir, 'test_results.csv')
with open(csv_output_path, 'w', newline='') as csvfile:
    fieldnames = ['test_loss', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    })

print(f"Risultati del testing salvati in: {csv_output_path}")


Creazione Grafici Testing

In [12]:
def plot_results(csv_file):
    results = {}
    with open(csv_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            for key, value in row.items():
                results[key] = float(value)

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 6))

    # Bar plot
    ax.bar(results.keys(), results.values(), color='skyblue')

    ax.set_ylabel('Value')
    ax.set_title('Testing Metrics')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.tight_layout()
    plt.show()

# Esempio di utilizzo
csv_file = os.path.join(checkpoint_dir, 'test_results.csv')
plot_results(csv_file)
